In [14]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from dataset.dataset import MSGDataModule
from models.FNO import FNO2d
from models.LightningModule import LitEstimator

# from pytorch_lightning.pytorch.callbacks import DeviceStatsMonitor
from train import config

In [15]:
print('asdf')

asdf


In [16]:
import torch
print(torch.__version__)

1.10.1+cu111


In [17]:
wandb_logger = WandbLogger(project="SIS_estimation")
dm = MSGDataModule(
    batch_size=config.BATCH_SIZE,
    num_workers=config.NUM_WORKERS,
    patch_size=config.INPUT_SIZE,
)

model = FNO2d(
    modes=(24, 24),
    input_channels=config.INPUT_CHANNELS,
    output_channels=config.OUTPUT_CHANNELS,
    channels=10,
)

estimator = LitEstimator(
    model=model,
    learning_rate=config.LEARNING_RATE,
)

trainer = Trainer(
    # profiler='simple',
    # callbacks=[DeviceStatsMonitor(cpu_stats=true)]
    num_sanity_val_steps=2,
    logger=wandb_logger,
    accelerator=config.ACCELERATOR,
    devices=config.DEVICES,
    min_epochs=config.MIN_EPOCHS,
    max_epochs=config.MAX_EPOCHS,
    precision=config.PRECISION,
    log_every_n_steps=1,
)

/scratch/snx3000/kschuurm/lightning19/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
/scratch/snx3000/kschuurm/lightning19/lib/python3.9/site-packages/lightning_fabric/plugins/environments/slurm.py:166: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /scratch/snx3000/kschuurm/lightning19/lib/python3.9/ ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [18]:
trainer.fit(model=estimator, train_dataloaders=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type                 | Params
------------------------------------------------
0 | model  | FNO2d                | 232 K 
1 | metric | RelativeSquaredError | 0     
------------------------------------------------
232 K     Trainable params
0         Non-trainable params
232 K     Total params
0.930     Total estimated model params size (MB)


set up batch generator for patches : 1.449 seconds


Sanity Checking: 0it [00:00, ?it/s]

RuntimeError: Tensors must have same number of dimensions: got 5 and 4

In [8]:
for x, y in dm.val_dataloader():
    break

In [9]:
x.shape

torch.Size([32, 11, 1, 64, 64])

In [10]:
y.shape

torch.Size([32, 1, 64, 64])